In [2]:
from sympy import *
import itertools as it
import numpy as np

# Strategic equivalence to weighted zero-sum games

Given a game $u$ find games $z, k$ such that $u = z + k$ with $z$ weighted zero-sum game and $k$ non-strategic game solving non-linear system. For details, see Obsidian note (local file)

`obsidian://open?vault=Work%20diary&file=Notes%2F2024-07-11%20Harmonic%20games%20and%20weighted%20zero%20sum%20games`

# Parameters

In [3]:
# Game skeleton

# numPlayers = 4

# skeleton = np.random.randint(2, 6, numPlayers)

#skeleton = [2, 3]
skeleton = [2,2,2]
#skeleton = [3,3]
skeleton

[2, 2, 2]

In [4]:
# N
numPlayers = len(skeleton)
players = range(numPlayers)

# -------------------------
# Number of action profiles
# -------------------------

# A
numPures = prod(skeleton)

# List of A_{-i} = for each player, number of action profiles of other players
numPuresMinus = [  int(numPures / skeleton[i]) for i in players ]

# -------------------------
# Number of payoff degrees of freedom
# -------------------------

# AN; number of payoff degrees of freedom
numPays = numPlayers * numPures

In [5]:
# RANDOM PAYOFF
# Payoff in usual Candogan format: flat list of size AN = numPays
# u = np.random.randint(-5, 5, numPays)
# u

In [6]:
# CHOSEN PAYOFF
# Payoff in usual Candogan format: flat list of size AN = numPays

#u = [17, -6, -7, 2, -1, -2, -7, 3, 3, 3, -2, -2]

# 223 generalized harmonic
# u = [-4, 12, -46, 34, -26, -49, -4, -1, 0, -2, 0, 3, 99/2, -31, -89, 1, 0, -1, -2, -2, 1, 0, -4, -5, -57/2, -24, 2, -4, -1, -5, 1, 2, -5, 3, -3, -5]

# 2x2 uniform harmonic --> always ~ to exact zero-sum game (as by Candogan's), since players have same # of strategies
# u = [4, -3, 0, 1, -1, 3, 3, -1]
# u  = [0, 4, 1, 3, 0, -1, 1, 2]
# u = [-2, 2, -2, 2, 1, 1, -1, -1]
# u = [-1, 0, -1, 0, -2, -2, 3, 3]

# 3x3 uniform harmonic --> always ~ to exact zero-sum game (as by Candogan's), since players have same # of strategies
# u = [4.66666666666667, 1.66666666666667, -7.33333333333333, 1.33333333333333, 0.333333333333333, -2.66666666666667, 3, -2, -2, -3.00000000000000, -3.00000000000000, 2, 1, -1, -2, 0, 2, -2]
# u = [0.666666666666667, -0.333333333333333, -0.333333333333333, -2.66666666666667, 0.333333333333333, 2.33333333333333, -1, 0, 1, -1.00000000000000, 1.00000000000000, 2, 3, 1, 0, -2, -2, -2]
# u = [2.33333333333333, 6.33333333333333, -0.666666666666667, 4.66666666666667, 0.666666666666667, 2.66666666666667, 3, 3, 2, -4, -8, -3, -3, 1, -3, 3, 3, 2]

# 2x2x2 uniform harmonic --> always ~ to exact zero-sum game (as by Candogan's), since players have same # of strategies
# u = [-2.00000000000000, -5.00000000000000, 1.00000000000000, 9.00000000000000, 2, 0, 1, 0, -1.00000000000000, 10.0000000000000, -3, 3, -2, -2, 2, 3, 2.00000000000000, 0, 2, 0, -2, -2, -3, 1]


# 2x2x2 zero-sum potential, no strict NE, 6 pure NE, 1 mixed NE
# 222 zero-sum
from aspera import utils
U = [ [0,0,0], [-1, -1, 2], [ -1, 2, -1 ], [ 2, -1, -1 ], [ 2, -1, -1 ], [ -1, 2, -1 ], [ -1, -1, 2 ], [ 0, 0, 0 ]  ]
u1, u2, u3 = utils.coords_points(U)
#u = u1 + u2 + u3

In [7]:
#u = [-7, -1, -5, 4, -3, -2, 3, -1, -7, 5, 1, 0, -3, 1, -5, 3, 3, -3, 1, 1, -2, 1, 1, 4]

In [8]:
# 33 Shapley RPS
#u = [0, 1, 0, 0, 0, 1, 1, 0, 0] + [0, 0, 1, 1, 0, 0, 0, 1, 0]

In [9]:
# 222 Jordan Matching Pennies
u = [1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1]

In [10]:
assert len(u) == numPays

Number of non-strategic dofs: each player has many dofs as number of action profiles of others; sum over those:

$$\sum_i \prod_{j\neq i}A_j = \sum_i \frac{A}{A_i}$$


In [11]:
# number of non-strategic payoff degrees of freedom
numPaysNS = sum(numPuresMinus)

In [12]:
numPures

8

In [13]:
numPuresMinus

[4, 4, 4]

In [14]:
numPays

24

In [15]:
numPaysNS

12

---
# Program

In [16]:
# Make symbols: methods to dynamically create as many as needed strings and Sympy symbols
# Shift is starting index; default 0

def make_strings(N,s, shift = 0):
	my_strings = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    my_strings.append(tmp_st)
	return my_strings

def make_symbols(N,s, shift = 0):
	my_symbols = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    globals()[tmp_st] = Symbol(tmp_st)
	    my_symbols.append(globals()[tmp_st])
	return my_symbols

In [17]:
# Pure actions: list of N lists, each with Ai elements; pure actions of each player

pures_play = [ make_strings(skeleton[i], f'a{i}', shift = 1) for i in players ]

In [18]:
pures_play

[['a01', 'a02'], ['a11', 'a12'], ['a21', 'a22']]

In [19]:
# Pure profiles; cartesian product of pure actions of each player
# Returns one list with A = numPures elements; each element is a tuple of strings

pures = list(it.product(*pures_play))

In [20]:
pures

[('a01', 'a11', 'a21'),
 ('a01', 'a11', 'a22'),
 ('a01', 'a12', 'a21'),
 ('a01', 'a12', 'a22'),
 ('a02', 'a11', 'a21'),
 ('a02', 'a11', 'a22'),
 ('a02', 'a12', 'a21'),
 ('a02', 'a12', 'a22')]

In [21]:
assert len(pures) == numPures

# Payoff of original game

In [22]:
# Pack flat payoff of sizer AN into a list of N lists, each of size A; player-specific payoffs
ui = [ u[ i*numPures : (i+1)*numPures] for i in players]
ui

[[1, 1, -1, -1, -1, -1, 1, 1],
 [1, -1, -1, 1, 1, -1, -1, 1],
 [-1, 1, -1, 1, 1, -1, 1, -1]]

In [23]:
# Build payoff dict matching pures with each player-specific payoff
u = [  dict(zip(pures , ui[i] ))  for i in players]

In [24]:
def print_payoff(payoff_dict, payoff_symbol):
    for i in players:
        for a in pures:
            print( f'{payoff_symbol}_{i}{a} = {payoff_dict[i][a]}' )
        print()

In [25]:
# Print payoff
print_payoff(u, 'u')

u_0('a01', 'a11', 'a21') = 1
u_0('a01', 'a11', 'a22') = 1
u_0('a01', 'a12', 'a21') = -1
u_0('a01', 'a12', 'a22') = -1
u_0('a02', 'a11', 'a21') = -1
u_0('a02', 'a11', 'a22') = -1
u_0('a02', 'a12', 'a21') = 1
u_0('a02', 'a12', 'a22') = 1

u_1('a01', 'a11', 'a21') = 1
u_1('a01', 'a11', 'a22') = -1
u_1('a01', 'a12', 'a21') = -1
u_1('a01', 'a12', 'a22') = 1
u_1('a02', 'a11', 'a21') = 1
u_1('a02', 'a11', 'a22') = -1
u_1('a02', 'a12', 'a21') = -1
u_1('a02', 'a12', 'a22') = 1

u_2('a01', 'a11', 'a21') = -1
u_2('a01', 'a11', 'a22') = 1
u_2('a01', 'a12', 'a21') = -1
u_2('a01', 'a12', 'a22') = 1
u_2('a02', 'a11', 'a21') = 1
u_2('a02', 'a11', 'a22') = -1
u_2('a02', 'a12', 'a21') = 1
u_2('a02', 'a12', 'a22') = -1



# Profiles of other players

In [26]:
# Pure profiles of other players
# Make list of N lists; the list pure_minus[i] contains the pure action profiles of players other than i
# Build taking the cartesian product of pure actions of all players other than i
# The size of the list pure_minus[i] is A_{-i} = \prod_{j \neq i} A_j

pures_minus = [ list( it.product( *( pures_play[:i] + pures_play[i+1:] ) ) ) for i in players ]

In [27]:
pures_minus

[[('a11', 'a21'), ('a11', 'a22'), ('a12', 'a21'), ('a12', 'a22')],
 [('a01', 'a21'), ('a01', 'a22'), ('a02', 'a21'), ('a02', 'a22')],
 [('a01', 'a11'), ('a01', 'a12'), ('a02', 'a11'), ('a02', 'a12')]]

In [28]:
numPuresMinus

[4, 4, 4]

In [29]:
for i in players:
    assert len(pures_minus[i]) == numPuresMinus[i]

# Non-strategic game

In [30]:
# Non-strategic payoff degrees of freedom; will build non-strategic payoff k out of these

# to be determined non-strategic payoff of player i (as many as action profiles of players -i)

n_sym = [make_symbols(numPuresMinus[i], f'alpha{i}', shift = 1) for i in players]
n_sym

[[alpha01, alpha02, alpha03, alpha04],
 [alpha11, alpha12, alpha13, alpha14],
 [alpha21, alpha22, alpha23, alpha24]]

In [31]:
# Given the non-strategic dofs, match them with the action profiles of other players

# Build list of N dictionary.
# The keys of the dictionary n_dicts[i] are action profiles a_{-i} of players -i (that is, list of tuples, elements of pures_minus[i] )
# The values of the dictionary n_dicts[i] are the non-strategic payoff of player i given the profile a_{-i}

# read dictionary as: when player -i plays a_{-i} , player i gets n_dicts[i][ a_{-i} ]

n_dicts = [ dict(zip(pures_minus[i], n_sym[i])) for i in players ]

In [32]:
n_dicts

[{('a11', 'a21'): alpha01,
  ('a11', 'a22'): alpha02,
  ('a12', 'a21'): alpha03,
  ('a12', 'a22'): alpha04},
 {('a01', 'a21'): alpha11,
  ('a01', 'a22'): alpha12,
  ('a02', 'a21'): alpha13,
  ('a02', 'a22'): alpha14},
 {('a01', 'a11'): alpha21,
  ('a01', 'a12'): alpha22,
  ('a02', 'a11'): alpha23,
  ('a02', 'a12'): alpha24}]

In [33]:
# Util to make (a_i, a_{-i}) given a_i and a_{-i} as tuple of strings (to be used as key for payoff dictionaries)
def make_a(ai, a_minus_i, i):
    l = list(a_minus_i)
    return tuple(l[:i] + [ai] + l[i:])

In [34]:
# Build full non-strategic payoff using non-strategic payoff degrees of freedom
# For each player, k_i does not depend on a_i but only on a_{-i}, so
# for each a_i, for each a_{-i}, build a = (a_i, a_{-i}) and assign k_i(a) = n_dicts[i][ a_{-i} ]

# list of dicts
k = []

for i in players:
    ki = {  }
    for ai in pures_play[i]:
        for a_minus_i in pures_minus[i]:
            a = make_a( ai, a_minus_i, i )
            ki[a] = n_dicts[i][a_minus_i]
    k.append(ki)

# Full payoff of non-strategic game, to be determined
k

[{('a01', 'a11', 'a21'): alpha01,
  ('a01', 'a11', 'a22'): alpha02,
  ('a01', 'a12', 'a21'): alpha03,
  ('a01', 'a12', 'a22'): alpha04,
  ('a02', 'a11', 'a21'): alpha01,
  ('a02', 'a11', 'a22'): alpha02,
  ('a02', 'a12', 'a21'): alpha03,
  ('a02', 'a12', 'a22'): alpha04},
 {('a01', 'a11', 'a21'): alpha11,
  ('a01', 'a11', 'a22'): alpha12,
  ('a02', 'a11', 'a21'): alpha13,
  ('a02', 'a11', 'a22'): alpha14,
  ('a01', 'a12', 'a21'): alpha11,
  ('a01', 'a12', 'a22'): alpha12,
  ('a02', 'a12', 'a21'): alpha13,
  ('a02', 'a12', 'a22'): alpha14},
 {('a01', 'a11', 'a21'): alpha21,
  ('a01', 'a12', 'a21'): alpha22,
  ('a02', 'a11', 'a21'): alpha23,
  ('a02', 'a12', 'a21'): alpha24,
  ('a01', 'a11', 'a22'): alpha21,
  ('a01', 'a12', 'a22'): alpha22,
  ('a02', 'a11', 'a22'): alpha23,
  ('a02', 'a12', 'a22'): alpha24}]

In [35]:
# Check that k has the right size
assert sum( [len(d) for d in k] ) == numPays

# Zero-sum weights

In [36]:
# Coefficients for weighted zero-sum game

lam = make_symbols(numPlayers, 'lambda', shift = 1)
lam

[lambda1, lambda2, lambda3]

# System

In [37]:
# Unknowns of system to solve: the weights (one per player) + the non-strategic dofs
dofs = lam + list(it.chain(*n_sym))
dofs

[lambda1,
 lambda2,
 lambda3,
 alpha01,
 alpha02,
 alpha03,
 alpha04,
 alpha11,
 alpha12,
 alpha13,
 alpha14,
 alpha21,
 alpha22,
 alpha23,
 alpha24]

In [38]:
# Number of unknowns of system to solve: the weights (one per player) + the non-strategic dofs
numDofs = numPlayers + numPaysNS
assert len(dofs) == numDofs

## System of non-linear equations

$$\sum_i \lambda_i \,  \big[u_i (\alpha) - k_i(\alpha) \big] = 0 \quad \text{for all } \alpha \in \mathcal{A}$$
- $u_i(\alpha)$ known coefficients
- $\lambda_i$ and $k_i(\alpha)$ unknowns

In [39]:
# --------------------------------------------------------------------------------------
eqs = [  sum( [ lam[i] * ( -u[i][a] + k[i][a] ) for i in players ] )  for a in pures  ]
# --------------------------------------------------------------------------------------

In [40]:
eqs

[lambda1*(alpha01 - 1) + lambda2*(alpha11 - 1) + lambda3*(alpha21 + 1),
 lambda1*(alpha02 - 1) + lambda2*(alpha12 + 1) + lambda3*(alpha21 - 1),
 lambda1*(alpha03 + 1) + lambda2*(alpha11 + 1) + lambda3*(alpha22 + 1),
 lambda1*(alpha04 + 1) + lambda2*(alpha12 - 1) + lambda3*(alpha22 - 1),
 lambda1*(alpha01 + 1) + lambda2*(alpha13 - 1) + lambda3*(alpha23 - 1),
 lambda1*(alpha02 + 1) + lambda2*(alpha14 + 1) + lambda3*(alpha23 + 1),
 lambda1*(alpha03 - 1) + lambda2*(alpha13 + 1) + lambda3*(alpha24 - 1),
 lambda1*(alpha04 - 1) + lambda2*(alpha14 - 1) + lambda3*(alpha24 + 1)]

In [ ]:
sol = nonlinsolve(eqs, dofs)

In [ ]:
dofs

In [ ]:
sol

In [ ]:
# select sols with lambda1 and lambda2 non-zero (if it exists)
extracted_sol = list(list(sol)[-1])

In [ ]:
extracted_sol

In [ ]:
# zip solution in dictionary with dofs
sol_dict = dict(zip(dofs, extracted_sol))

In [ ]:
sol_dict

In [ ]:
# fix remaining dofs; to avoid generating zero lambda, generate dictionary with dofs keys and with positive random values
fix_remaining_dofs = dict(zip(dofs, np.random.randint(1, 5, numDofs)))

In [ ]:
fix_remaining_dofs

In [ ]:
# fix remaining dofs to random number by subbing in sol_dict
sol_dict_instance = { key : sol_dict[key].subs(fix_remaining_dofs) for key in sol_dict}

In [ ]:
sol_dict_instance

In [ ]:
# Sometimes solution returns finite set; convert to float
for key in sol_dict_instance:
    value = sol_dict_instance[key]
    if type(value) == FiniteSet:
        sol_dict_instance[key] = value.args[0]
    elif type(value) == Interval:
        print(f'\nThere is interval solution: {key} in {value}\n')

sol_dict_instance

In [ ]:
# Manually fix solution in interval if needed
#sol_dict_instance[lambda2] = 1
#sol_dict_instance

In [ ]:
# util to replace solution dictionary in target dictionary

def replace_in_dict(target_dict, sol_dict):
    return { key : target_dict[key].subs(sol_dict) for key in target_dict }

In [ ]:
k_sol = [replace_in_dict(ki, sol_dict_instance) for ki in k]

In [ ]:
# Solution : full payoff of non-strategic game
k_sol

In [ ]:
# Solution: zero-sum weights
lam_sol = [l.subs(sol_dict_instance) for l in lam]
lam_sol

# Build weighted zero sum game
$$u = z+k$$ so $$z = u-k$$

In [ ]:
# Weighted zero-sum game full payoff dictionary

z = [ { a: u[i][a] - k_sol[i][a] for a in pures } for i in players   ]

z

In [ ]:
# Assert is weighted zero-sum
for a in pures:
    weighted_sum = sum( [ lam_sol[i] * z[i][a] for i in players ] )
    assert weighted_sum < 1e-12

# Print results

In [ ]:
print(f'\nGame: {skeleton}')
print(f'\nu = z + k')
print('\nu original game')
print('z weighetd zero-sum game')
print('k non-strategic game')
print(f'\nWeights = {lam_sol} such that sum_i λ_i z_i = 0\n' )

sol_exist = True
for l in lam_sol:
    if l == 0:
        sol_exist = False
        print('Solution not found')
        break
if sol_exist: print('Solution found')

In [ ]:
print_payoff(u, 'u')
print_payoff(z, 'z')
print_payoff(k_sol, 'k')

# Findings
Being $\sim$ means being strategically equivalent to a weighted zero-sum game.

## Random
### 2-player
- random $2\times2$ seems always $\sim$
- random $2$-player game other than $2\times2$  seems never $\sim$

### 3-player
- random $3$-player seems often $\sim$

### 4-player
- random $4$-player seems never $\sim$

## Harmonic (generalized, N player)
- **proved: always ~**

# End